<h1>Generate Answers from MedVet with API</h2>
In this notebook we generate the answers to the described cases with the API of Medvet.
The creation will be done one time with a combined answer of an image from LLaVA-Med and the LlaMA 2 RAG system and one time with a separate output.

Specifications:

---------------------------
<p>Used model in MedVet: LlaMA 2 7b-chat with 8bit quantization as .gguf format </p>
<p>used prompt template: "Casesolver"</p>

In [1]:
import requests
import base64
from PIL import Image
from io import BytesIO
import json

In [2]:
ip_address_llava = "" #Have to be updated after start of EC2 instance
ip_address_medvet = "127.0.0.1:8000"

url = f"http://{ip_address_medvet}/generate"
clear_url_llava = f"http://{ip_address_llava}/clearchat"
clear_url_llama = f"http://{ip_address_medvet}/clearchat"

In [3]:
def clearChat():
    try:
        if ip_address_llava != "":
            requests.post(clear_url_llava)
        if ip_address_medvet != "":
            requests.post(clear_url_llama)
    except Exception as e:
        print( f"Failed to connect to AWS:{e}")

Generation method for llama3

In [20]:
def getAnswerMedVet(case,image_name,combined:bool,chaintype):
    output = ""
    data = {
                "prompt": [{"role":"assistant","content": "How may I assist you?"},{"role":"user","content": f"{case}"}],
                "agent_id": "validation_02",
                "ip_address_llava": "",
                "img": "",
                "chaintype":chaintype,
                "display_combined": combined,
                "mode_assistant": "simple_kb" ,
                "llm_template_name": "llama3_plain_meta",
                "use_rag": 1,
                "temperature": 0.7,
                "max_new_tokens": 1024
            }
    headers = {'Content-type': 'application/json', 'Accept': 'application/json'}
    try:
        response = requests.put(url, data=json.dumps(data), headers=headers)
        print(f"Response: {response}")
        response = response.json()
        output = response

    except Exception as e:
        output = f"Failed to connect to AWS:{e}"
    return output

Generation method for LLaMA2

In [13]:
def getAnswerMedVet(case,image_name,combined:bool,chaintype):
    output = ""
    data = {
                "prompt": [{"role":"assistant","content": "How may I assist you?"},{"role":"user","content": f"{case}"}],
                "agent_id": "validation_02",
                "ip_address_llava": "",
                "img": "",
                "chaintype":chaintype,
                "display_combined": combined,
                "mode_assistant": "simple_kb" ,
                "llm_template_name": "llama2_plain" if chaintype == "plain" else "llama2_RAG",
                "use_rag": 1,
                "temperature": 0.7,
                "max_new_tokens": 1024
            }
    headers = {'Content-type': 'application/json', 'Accept': 'application/json'}
    try:
        response = requests.put(url, data=json.dumps(data), headers=headers)
        print(f"Response: {response}")
        response = response.json()
        output = response

    except Exception as e:
        output = f"Failed to connect to AWS:{e}"
    return output

In [17]:
import csv
import pandas as pd

lines_newCSV = []
df = pd.read_csv('Questions with answers.csv')

line_count = 0

for index, row in df.iterrows():
    answer = getAnswerMedVet(row['Question'],"",0,"plain")
    answer_rag = getAnswerMedVet(row['Question'],"",0,"rag")
    lines_newCSV.append({
        "Number":line_count,
        "Question": row['Question'],
        "Solution": row['Answer'],
        "Answer Medvet plain": answer['answer_llama']['answer'],
        "Answer Medvet rag": answer_rag['answer_llama']['answer'],
        })
    clearChat()
    line_count +=1


Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>


In [ ]:
with open('Results/EvaluationKB2/EvaluationKB_LLaMA2_Finetuned.csv', 'w', newline='') as csvfile:
    fieldnames = ['Number', "Question","Solution","Answer Medvet plain","Answer Medvet rag"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(lines_newCSV)

In [21]:
import csv
import pandas as pd

lines_newCSV = []
df = pd.read_csv('Questions with answers.csv')

line_count = 0

for index, row in df.iterrows():
    answer = getAnswerMedVet(row['Question'],"",0,"plain")
    lines_newCSV.append({
        "Number":line_count,
        "Question": row['Question'],
        "Solution": row['Answer'],
        "Answer Medvet plain": answer['answer_llama']['answer'],
        })
    clearChat()
    line_count +=1

Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>


In [22]:
with open('Results/EvaluationKB2/EvaluationKB_LLaMA3_Finetuned.csv', 'w', newline='') as csvfile:
    fieldnames = ['Number', "Question","Solution","Answer Medvet plain"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(lines_newCSV)

<h1>Separate</h1>